In [1]:
import tensorflow as tf
from tensorflow.keras import models , layers

In [2]:
IMAGE_SIZE=256
BATCH_SIZE=8
CHANNELS=3

In [3]:
dataset =tf.keras.preprocessing.image_dataset_from_directory(
    "Grape",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 9027 files belonging to 4 classes.


In [4]:
class_names = dataset.class_names
class_names

['Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy']

In [5]:
len(dataset)

1129

In [6]:
def get_dataset_partitions_tf(ds,train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):

  ds_size =len(ds)

  if shuffle:
    ds=ds.shuffle(shuffle_size)

  train_size = int(train_split*ds_size)
  val_size = int(val_split*ds_size)

  train_ds = ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
  test_ds=ds.skip(train_size).skip(val_size)


  return train_ds ,val_ds, test_ds

In [7]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [8]:
len(train_ds)

903

In [9]:
len(val_ds)

112

In [10]:
len(test_ds)

114

In [11]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)

])

In [13]:
data_augmentation = tf.keras.Sequential([
     layers.RandomFlip("horizontal_and_vertical"),
     layers.RandomRotation(0.2),
])

In [14]:
input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes=4
model1=models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Flatten(),

    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])
model1.build(input_shape=input_shape)

c:\Users\rekha\OneDrive\Desktop\Plant Disease Detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [15]:
model1.summary

<bound method Model.summary of <Sequential name=sequential_2, built=True>>

In [16]:
model1.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [18]:
checkpoint = ModelCheckpoint('./grape_model.keras',
    verbose=1,
    monitor='val_accuracy',
    save_best_only=True,
    mode='auto'
)

In [19]:
EPOCHS=10
history=model1.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

Epoch 1/10
903/903 ━━━━━━━━━━━━━━━━━━━━ 0s 594ms/step - accuracy: 0.4801 - loss: 1.0686
Epoch 1: val_accuracy improved from -inf to 0.57478, saving model to ./grape_model.keras
903/903 ━━━━━━━━━━━━━━━━━━━━ 703s 717ms/step - accuracy: 0.4803 - loss: 1.0683 - val_accuracy: 0.5748 - val_loss: 1.8494
Epoch 2/10
903/903 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/step - accuracy: 0.8134 - loss: 0.4723
Epoch 2: val_accuracy improved from 0.57478 to 0.81473, saving model to ./grape_model.keras
903/903 ━━━━━━━━━━━━━━━━━━━━ 511s 565ms/step - accuracy: 0.8134 - loss: 0.4723 - val_accuracy: 0.8147 - val_loss: 0.5033
Epoch 3/10
903/903 ━━━━━━━━━━━━━━━━━━━━ 0s 619ms/step - accuracy: 0.9133 - loss: 0.2376
Epoch 3: val_accuracy did not improve from 0.81473
903/903 ━━━━━━━━━━━━━━━━━━━━ 574s 634ms/step - accuracy: 0.9133 - loss: 0.2376 - val_accuracy: 0.6618 - val_loss: 1.5084
Epoch 4/10
903/903 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step - accuracy: 0.9339 - loss: 0.1767
Epoch 4: val_accuracy improved from 0.81473 to 0.9229

In [20]:
scores=model1.evaluate(test_ds)

114/114 ━━━━━━━━━━━━━━━━━━━━ 143s 155ms/step - accuracy: 0.9794 - loss: 0.0580


In [21]:
history.params

{'verbose': 1, 'epochs': 10, 'steps': 903}

In [22]:
history.history.keys()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])

In [23]:
history.history['accuracy']

[0.6385926008224487,
 0.8343260884284973,
 0.9188253283500671,
 0.9351710677146912,
 0.9516553282737732,
 0.9584429860115051,
 0.964676558971405,
 0.964953601360321,
 0.976035475730896,
 0.9747887253761292]

In [24]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

In [25]:
val_acc

[0.5747767686843872,
 0.8147321343421936,
 0.6618303656578064,
 0.9229910969734192,
 0.9229910969734192,
 0.9375,
 0.9319196343421936,
 0.9084821343421936,
 0.9709821343421936,
 0.9821428656578064]

In [26]:
import numpy as np
for images_batch,label_batch in test_ds.take(1):
  first_image = images_batch[0].numpy().astype('uint8')
  first_label = label_batch[0].numpy()

  print('first image to predict:')

  #print("first image's actual label:",first_label)
  print("first image's actual label:",class_names[first_label])
  batch_prediction = model1.predict(images_batch)
  print(batch_prediction[0])
  print(np.argmax(batch_prediction[0]))
  print(class_names[np.argmax(batch_prediction[0])])




first image to predict:
first image's actual label: Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[2.052342e-02 9.958733e-05 9.777624e-01 1.614606e-03]
2
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)


In [27]:
def predict(model,img):
  img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
  img_array = tf.expand_dims(img_array,0)# Create a batch

  predictions = model.predict(img_array)

  predicted_class = class_names[np.argmax(predictions[0])]
  confidence = round(100* (np.max(predictions[0])),2)
  return predicted_class, confidence

In [28]:

for images, labels in test_ds.take(1):
  for i in range(9):

    predicted_class, confidence=predict(model1,images[i].numpy())
    print(predicted_class)
    print(confidence)
    actual_class = class_names[labels[i]]
    print(actual_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 857ms/step
Grape___Black_rot
99.98
Grape___Black_rot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
99.5
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Grape___healthy
99.8
Grape___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Grape___healthy
100.0
Grape___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Grape___Esca_(Black_Measles)
99.99
Grape___Esca_(Black_Measles)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Grape___healthy
100.0
Grape___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Grape___Esca_(Black_Measles)
100.0
Grape___Esca_(Black_Measles)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Grape___Esca_(Black_Measles)
99.99
Grape___Esca_(Black_Measles)


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 8 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/